In [1]:
import vlm_answers_gen as vlm

/opt/miniconda3/envs/acc_vis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'LlavaMiniLlamaForCausalLM' from 'transformers' (/opt/miniconda3/envs/acc_vis/lib/python3.11/site-packages/transformers/__init__.py)

In [2]:
pip install git+https://github.com/deepseek-ai/Janus

  Cloning https://github.com/deepseek-ai/Janus to /private/var/folders/nw/ssdc3p8s1pqbp4lsgsj33ys00000gp/T/pip-req-build-3otb0xgx
  Running command git clone --filter=blob:none --quiet https://github.com/deepseek-ai/Janus /private/var/folders/nw/ssdc3p8s1pqbp4lsgsj33ys00000gp/T/pip-req-build-3otb0xgx
  Resolved https://github.com/deepseek-ai/Janus to commit 1daa72fa409002d40931bd7b36a9280362469ead
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for janus: filename=janus-1.0.0-py3-none-any.whl size=85499 sha256=85dc39fa9b1d9b871f7edb8c54e4428b9ee5f8453ff1da4f74ae963458b0d41c
  Stored in directory: /private/var/folders/nw/ssdc3p8s1pqbp4lsgsj33ys00000gp/T/pip-ephem-wheel-cache-2kqraupt/wheels/d4/4f/75/619703dcd03931e7d82

In [2]:
csv_file = 'partition_6_9.csv'

In [ ]:
# model_name = "deepseek-ai/Janus-Pro-7B"
model_name = "ICTNLP/llava-mini-llama-3.1-8b"
processor, model = vlm.load_model(model_name, vlm.LlavaProcessor, vlm.LlavaMiniLlamaForCausalLM)

generation_kwargs = {
    "max_new_tokens": 1024,
    "num_beams": 1,
    "do_sample": False
}

AttributeError: module 'vlm_answers_gen' has no attribute 'LlavaMiniLlamaForCausalLM'

In [ ]:
results = vlm.process_questions(csv_file, model, processor, generation_kwargs, "test_vlm_output.csv")


In [ ]:
vlm.process_questions(, None, None, None, )

Processing Questions: 22question [00:00, 689.00question/s]

Checkpoint saved to test_vlm_output.csv
Checkpoint saved to test_vlm_output.csv


[{'path': 'img/test/008.jpg',
  'question': 'Where is the toilet positioned with respect to the room?',
  'true_answer': 'On the left side of the room',
  'type_question': 'qualitative',
  'answer': '\n    You will be provided an image and a question about the image.\n    You will also be told if the answer expected is a qualitative or quantitative answer.\n    Please provide an answer to the question. \n\n    Example 1:\n    Question: "How far is the yellow finger from the silver can?"\n    Expected Answer Type: Quantitative\n    Output: "The yellow finger is one and a half meters away from the silver can"\n\n    Example 2:\n    Question: "What is next to the blue sofa?"\n    Expected Answer Type: Qualitative\n    Output: "The yellow chair is next to the blue sofa"\n\n    Your task: Provide an answer to the question. No greetings or comments. Just provide the answer.\n    Question: Where is the toilet positioned with respect to the room?\n    Question Type: qualitative\n    '},
 {'pat

In [3]:
import argparse
import torch
import numpy as np
from transformers import AutoConfig, AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from PIL import Image

def load_model(model_path: str = "deepseek-ai/Janus-1.3B"):
    """
    Loads the Janus model and processor.
    """
    config = AutoConfig.from_pretrained(model_path)
    # For improved performance or if you get CUDA-related errors:
    # config.language_config._attn_implementation = 'eager'
    language_config = config.language_config
    language_config._attn_implementation = 'eager'

    # Load the base model
    vl_gpt = AutoModelForCausalLM.from_pretrained(
        model_path,
        language_config=language_config,
        trust_remote_code=True
    )

    # Move model to GPU if available, otherwise CPU
    if torch.cuda.is_available():
        vl_gpt = vl_gpt.to(torch.bfloat16).cuda()
    else:
        vl_gpt = vl_gpt.to(torch.float16)

    # Load the processor
    vl_chat_processor = VLChatProcessor.from_pretrained(model_path)

    return vl_gpt, vl_chat_processor

Python version is above 3.10, patching the collections module.


/opt/miniconda3/envs/acc_vis/lib/python3.11/site-packages/transformers/models/auto/image_processing_auto.py:517: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


In [ ]:
print("Loading model...")
vl_gpt, vl_chat_processor = load_model()

Loading model...


In [ ]:
@torch.inference_mode()
def ask_janus(
    vl_gpt: MultiModalityCausalLM,
    vl_chat_processor: VLChatProcessor,
    image_path: str,
    question: str,
    seed: int = 42,
    top_p: float = 0.95,
    temperature: float = 0.1
) -> str:
    """
    Given an image path and a question, returns the answer predicted by the Janus model.
    """
    # Set random seeds for reproducibility
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    # Read and prepare the image
    pil_image = Image.open(image_path).convert("RGB")

    # Build conversation data
    conversation = [
        {
            "role": "User",
            "content": f"<image_placeholder>\n{question}",
            "images": [pil_image],
        },
        {"role": "Assistant", "content": ""},
    ]

    # Move data to correct device / dtype
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float16

    # Process input (tokenization, image features, etc.)
    prepare_inputs = vl_chat_processor(
        conversations=conversation, images=[pil_image], force_batchify=True
    ).to(device, dtype=dtype)

    # Convert tokens + embeddings for the model
    inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

    # Generate the answer
    tokenizer = vl_chat_processor.tokenizer
    outputs = vl_gpt.language_model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=prepare_inputs.attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=(temperature != 0.0),
        use_cache=True,
        temperature=temperature,
        top_p=top_p,
    )

    # Decode model output into text
    answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
    return answer